# ID2214/FID3214 Assignment 3 Group no. [2]
### Project members: 
[Farrokh Bolandi, bolandi@kth.se]
[Ezio Cristofoli, ezioc@kth.se]
[Abyel Tesfay, Abyel@kth.se]


### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from math import log2

## Reused functions from Assignment 1

In [ ]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_column_filter(df):
    new_df = df.copy()
    new_df = new_df.filter([e for e in new_df.columns if new_df[e].nunique() > 1 or e in ['CLASS', 'ID']], axis=1)
    return new_df, list(new_df.columns)


def create_imputation(df):
    new_df = df.copy()
    imputation = {}
    for e in new_df.columns:
        if e not in ['CLASS', 'ID']:
            # numerical values
            if new_df[e].dtypes in ["float64", "int64"]:
                if new_df[e].nunique() < 1:
                    new_df[e].fillna(0, inplace=True)
                new_df[e].fillna(new_df[e].mean(), inplace=True)
                imputation[e] = new_df[e].mean()
            # categorical or object values
            else:
                # if it is an object
                if new_df[e].dtypes == "object":
                    new_df[e] = new_df[e].astype('category')
                if new_df[e].nunique() < 1:
                    new_df[e].fillna(new_df[e][0], inplace=True)
                new_df[e].fillna(new_df[e].mode()[0], inplace=True)
                imputation[e] = new_df[e].mode()[0]
    return new_df, imputation


def create_one_hot(df):
    new_df = df.copy()
    handle = new_df.filter([e for e in new_df.columns if e not in ['CLASS', 'ID']], axis=1)
    one_hot = {}
    for e in handle.columns:
        #print(new_df[e].dtypes.name)
        if new_df[e].dtypes.name == 'category':
            features = np.sort(handle[e].unique())  # 3
            for i in features:
                new_df[e + "-" + str(i)] = [1.0 if x == i else 0.0 for x in handle[e]]
                new_df[e + "-" + str(i)].astype('float')  # 4
            one_hot[e] = features
            new_df.drop(e, axis=1, inplace=True)  # 5
    return new_df, one_hot


def apply_column_filter(df, column_filter):
    new_df = df.copy()
    new_df = new_df.filter(items=column_filter, axis=1)
    return new_df


def apply_imputation(df, imputation):
    new_df = df.copy()
    new_df.fillna(imputation, inplace=True)
    return new_df


def apply_one_hot(df, one_hot):
    new_df = df.copy()
    for e in new_df.columns:
        if e in one_hot and e not in ['CLASS', 'ID']:
            for i in one_hot[e]:
                new_df[e + "-" + str(i)] = [1.0 if x == i else 0.0 for x in new_df[e]]
                new_df[e + "-" + str(i)].astype('float')  # 4
            new_df.drop(e, axis=1, inplace=True)  # 5
    return new_df


def accuracy(df, correctlabels):
    highest_probability = df.idxmax(axis=1)
    correct_occurances = 0
    for correct_label, predicted_label in zip(correctlabels, highest_probability):
        if correct_label == predicted_label:
            correct_occurances += 1

    return correct_occurances/df.index.size


def brier_score(df, correctlabels):
    squared_sum = 0
    row = 0
    for label in correctlabels:
        i = np.where(df.columns == label)[0]
        for col in df.columns:
            squared_sum += (1 - df.loc[row, label]
                            )**2 if label == col else df.loc[row, col]**2
        row += 1
    return squared_sum/df.index.size


def auc(df, correctlabels):
    auc = 0
    for col in df.columns:
        df2 = pd.concat(
            [df[col], pd.Series(correctlabels.astype('category'), name='correct')], axis=1)
        # get dummies for correct labels and sort descending
        df2 = pd.get_dummies(df2.sort_values(col, ascending=False))

        # move col to first for easier total tp and fp calculation
        tmp = df2.pop('correct_'+str(col))
        # get the col frequency for calculating weighted AUCs
        col_frequency = tmp.sum()/tmp.index.size
        df2.insert(1, tmp.name, tmp)
        scores = {}
        # populate the scores dictionary for column i.e. key=score, value=[tp_sum, fp_sum]
        for row in df.index:
            key = df2.iloc[row, 0]
            current = np.zeros(2, dtype=np.uint) if scores.get(
                key) is None else scores[key]
            to_add = np.array([1, 0]) if df2.iloc[row,
                                                  1] == 1 else np.array([0, 1])
            scores[key] = current+to_add

        # calculate auc based on scores
        cov_tp = 0
        column_auc = 0
        tot_tp = 0
        tot_fp = 0
        # calculate total tp and fp
        for value in scores.values():
            tot_tp += int(value[0])
            tot_fp += int(value[1])

        # same algorithm as in the lecture (bad naming though)
        for i in scores.values():
            if i[1] == 0:
                cov_tp += i[0]
            elif i[0] == 0:
                column_auc += (cov_tp/tot_tp)*(i[1]/tot_fp)
            else:
                column_auc += (cov_tp/tot_tp) * \
                    (i[1]/tot_fp)+(i[0]/tot_tp)*(i[1]/tot_fp)/2
                cov_tp += i[0]

        auc += col_frequency*column_auc

    return auc

## 1. Define the class RandomForest

In [17]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):

class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        # 1
        new_df = df.copy()
        new_df, self.column_filter = create_column_filter(new_df)
        new_df, self.imputation = create_imputation(new_df)
        new_df, self.one_hot = create_one_hot(new_df)

        # 2
        # .astype('category').cat.categories.to_numpy()
        self.labels = np.array(new_df["CLASS"].values)
        new_df.drop(["CLASS"], axis=1, inplace=True)
        instances = np.array(new_df.values)

        # 3
        self.model = []
        rng = np.random.default_rng()
        for i in range(no_trees):
            features = int(log2(len(new_df.columns)))
            dt = DecisionTreeClassifier(max_features=features)                                           # 1. generate decision tree       
            bootstrap_indexes = rng.choice(np.array(new_df.index), size=len(new_df.index), replace=True) # 2. create list of random indexes for random sampling with replacement
            bootstrap_sample = [instances[e] for e in bootstrap_indexes]                                 # 3. create single bootstrap sample for model
            bootstrap_labels = [self.labels[e] for e in bootstrap_indexes]
            self.model.append(dt.fit(bootstrap_sample, bootstrap_labels))                                # 4. train base model with bootstrap instances

    def predict(self, df):

        # 1
        new_df = df.copy()
        new_df.drop(["CLASS"], axis=1, inplace=True)
        new_df = apply_column_filter(new_df, self.column_filter)
        new_df = apply_imputation(new_df, self.imputation)
        new_df = apply_one_hot(new_df, self.one_hot)

        # 2
        y_predictions = pd.DataFrame(0, index=df.index, columns=np.unique(self.labels))  # before: np.unique(self.labels)
        for i, tree in enumerate(self.model):
            prediction_i = tree.predict_proba(new_df.values)
            y_predictions = y_predictions.add(prediction_i)
        pred = y_predictions/len(self.model)
        return pred

        
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 



In [18]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.32 s.
Testing time: 0.10 s.
Accuracy: 0.9186
AUC: 0.9904
Brier score: 0.1744


In [19]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0222


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [20]:
class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None

    def fit(self, df, no_trees=100):
        # 1
        new_df = df.copy()
        new_df, self.column_filter = create_column_filter(new_df)
        new_df, self.imputation = create_imputation(new_df)
        new_df, self.one_hot = create_one_hot(new_df)

        # 2
        self.labels = np.array((new_df["CLASS"].values))          
        new_df.drop(["CLASS"], axis=1, inplace=True)
        instances = np.array(new_df.values)

        # 3
        self.model = []
        rng = np.random.default_rng()
        for i in range(no_trees):
            features = int(log2(len(new_df.columns)))
            dt = DecisionTreeClassifier(max_features=features)                                              # 1. generate decision tree       #int(log2(len(df.columns))
            bootstrap_indexes = rng.choice(np.array(new_df.index), size=len(new_df.index), replace=True)    # 2. create list of random indexes for random sampling with replacement
            bootstrap_sample = [instances[e] for e in bootstrap_indexes]                                    # 3. create single bootstrap sample for model
            bootstrap_labels = [self.labels[e] for e in bootstrap_indexes]
            self.model.append(dt.fit(bootstrap_sample, bootstrap_labels))                                   # 4. train base model with bootstrap instances
        
    def predict(self, df):

        # 1
        new_df = df.copy()
        new_df.drop(["CLASS"], axis=1, inplace=True)
        new_df = apply_column_filter(new_df, self.column_filter)
        new_df = apply_imputation(new_df, self.imputation)
        new_df = apply_one_hot(new_df, self.one_hot)
        
        class_labels = np.unique(sorted(self.labels))
        mapping = {instance: i for i, instance in enumerate(class_labels)}

        # 2
        y_predictions = pd.DataFrame(0, index=df.index, columns=class_labels)  
        for i, tree in enumerate(self.model):
            prediction_i = tree.predict_proba(new_df.values)
            for col in range(prediction_i.shape[1]):
                current_label = tree.classes_[col]
                correct_col = mapping.get(current_label)
                y_predictions.iloc[:,correct_col] += prediction_i[:, col]      
        pred = y_predictions/len(self.model)
        return pred

# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 



In [21]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.28 s.
Testing time: 0.25 s.
Accuracy: 0.9488
AUC: 0.9725
Brier score: 0.0976


## 2b. Estimate predictive performance using out-of-bag predictions

In [26]:
class RandomForest:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.model = None
        self.oob_acc = 0.0

    def fit(self, df, no_trees=100):
        new_df = df.copy()
        new_df, self.column_filter = create_column_filter(new_df)
        new_df, self.imputation = create_imputation(new_df)
        new_df, self.one_hot = create_one_hot(new_df)

        self.labels = np.array(new_df["CLASS"].values)         
        new_df.drop(["CLASS"], axis=1, inplace=True)
        instances = np.array(new_df.values)

        self.model = []
        rng = np.random.default_rng()
        
        ## task 2b
        c_labels = np.unique(sorted(self.labels))   #1 for 2b
        mapping = {instance: i for i, instance in enumerate(c_labels)}
        oob_predictions = pd.DataFrame(0, index=new_df.index, columns=c_labels)  
        oob_vector = pd.Series(0, index=new_df.index)
        
        for i in range(no_trees):
            features = int(log2(len(new_df.columns)))
            tree = DecisionTreeClassifier(max_features=features)                                              # 1. generate decision tree       #int(log2(len(df.columns))
            bootstrap_indexes = rng.choice(np.array(new_df.index), size=len(new_df.index), replace=True)    # 2. create list of random indexes for random sampling with replacement
            bootstrap_sample = [instances[e] for e in bootstrap_indexes]                                   # 3. create single bootstrap sample for model
            bootstrap_labels = [self.labels[e] for e in bootstrap_indexes]
            self.model.append(tree.fit(bootstrap_sample, bootstrap_labels))    
                               # 4. train base model with bootstrap instances
            ## task 2b: for oob predictions
            #bootstrap_sample = np.array(bootstrap_sample)
            
            # try instead: use difference to get all out-of-bag values, then do predict with thoose rows and add them to the prediction
            for indx in new_df.index: 
                if indx not in bootstrap_indexes:
                    #print(str(indx) + " not in bootstrap indexes!")
                    current_instance = np.array(instances[indx]).reshape(1, -1)
                    X = tree.predict_proba(current_instance)
                    for col in range(X.shape[1]):
                        current_label = tree.classes_[col]
                        correct_col = mapping.get(current_label)
                        oob_predictions.iloc[indx,correct_col] += X[0][col]   
                        oob_vector[indx] += 1
                    continue
        print("tree done")
        
        oob_predictions = oob_predictions.div(oob_vector, axis=0)
        self.oob_acc = accuracy(oob_predictions, df["CLASS"])
    
    def predict(self, df):

        new_df = df.copy()
        new_df.drop(["CLASS"], axis=1, inplace=True)
        new_df = apply_column_filter(new_df, self.column_filter)
        new_df = apply_imputation(new_df, self.imputation)
        new_df = apply_one_hot(new_df, self.one_hot)
        
        class_labels = np.unique(sorted(self.labels))
        mapping = {instance: i for i, instance in enumerate(class_labels)}

        y_predictions = pd.DataFrame(0, index=df.index, columns=class_labels)  
        for i, tree in enumerate(self.model):
            prediction_i = tree.predict_proba(new_df.values)
            for col in range(prediction_i.shape[1]):
                current_label = tree.classes_[col]
                correct_col = mapping.get(current_label)
                y_predictions.iloc[:,correct_col] += prediction_i[:, col]      
        pred = y_predictions/len(self.model)
        return pred

# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector


In [28]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

tree done
Training time: 30.13 s.
OOB accuracy: 0.9488
Testing time: 0.33 s.
Accuracy: 0.9488
AUC: 0.9713
Brier score: 0.0998


In [16]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

tree done
Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.